In [254]:
import numpy
import pandas
from pandas import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')

from dataloader import RawDataLoader, DataLoader

raw_loader = RawDataLoader()

In [346]:
combined_data = []

for group in range(1, 11):
    data_raw = raw_loader.load_eval(str(group))

    for idx in data_raw.index:
        data = data_raw.ix[idx]
        try:
            data_sensor = raw_loader.load_sensor_data(data['File name'])
        except:
            continue

        posture = data[' Input posture']
        if posture == 'Both':
            posture = 1
        elif posture == 'Right':
            posture = 2
        else:
            posture = 0
            
        age = data[' Age']
        
        combined_data.append([posture,
                    age,
                    data_sensor.acc_x.var(),
                    data_sensor.acc_y.var(),
                    data_sensor.acc_z.var(),
                    data_sensor.azim.var(),
                    data_sensor.roll.var(),
                    data_sensor.gyro_y.var(),
                    data_sensor.gyro_z.var(),
                    data_sensor.acc_x.mean(),
                    data_sensor.acc_y.mean(),
                    data_sensor.acc_z.mean(),
                    data_sensor.azim.mean(),
                    data_sensor.roll.mean(),
                    data_sensor.gyro_y.mean(),
                    data_sensor.gyro_z.mean()])

In [347]:
df = DataFrame(combined_data, columns=['posture', 'age', 'acc_x', 'acc_y', 'acc_z', 'azim', 'roll', 'gyro_y', 'gyro_z', 'macc_x', 'macc_y', 'macc_z', 'mazim', 'mroll', 'mgyro_y', 'mgyro_z'])
df.to_csv('data/train.csv')

In [348]:
train_data = read_csv('data/train.csv')

trainY = train_data['posture'].values
trainX = train_data.drop('posture', 1)

In [349]:
test_data = []
for group in range(1, 3):
    data_raw = raw_loader.load_eval('test' + str(group))

    for idx in data_raw.index:
        data = data_raw.ix[idx]
        data_sensor = raw_loader.load_sensor_data(data['File name'])

        posture = data[' Input posture']
        if posture == 'Both':
            posture = 1
        elif posture == 'Right':
            posture = 2
        else:
            posture = 0

        age = data[' Age']

        test_data.append([posture,
                          age,
                        data_sensor.acc_x.var(),
                        data_sensor.acc_y.var(),
                        data_sensor.acc_z.var(),
                        data_sensor.azim.var(),
                        data_sensor.roll.var(),
                        data_sensor.gyro_y.var(),
                        data_sensor.gyro_z.var(),
                        data_sensor.acc_x.mean(),
                        data_sensor.acc_y.mean(),
                        data_sensor.acc_z.mean(),
                        data_sensor.azim.mean(),
                        data_sensor.roll.mean(),
                        data_sensor.gyro_y.mean(),
                        data_sensor.gyro_z.mean()])

df = DataFrame(test_data, columns=['posture', 'age', 'acc_x', 'acc_y', 'acc_z', 'azim', 'roll', 'gyro_y', 'gyro_z', 'macc_x', 'macc_y', 'macc_z', 'mazim', 'mroll', 'mgyro_y', 'mgyro_z'])
df.to_csv('data/test.csv')

test_data = read_csv('data/test.csv')

testY = test_data['posture'].values
testX = test_data.drop('posture', 1)

In [358]:
rf = RandomForestClassifier(n_estimators=100, max_depth=30, max_leaf_nodes=100)
#svc = SVC(kernel="linear", C=0.025)
rf.fit(trainX, trainY)

predTestY = rf.predict(testX)
accuracyTest = float(np.sum(predTestY == testY)) / predTestY.shape[0]

print "testSet accuracy: ", accuracyTest*100, "%"

testSet accuracy:  64.2556179775 %


In [364]:
print "Original  :"
print testY[1200:1400]
print "Predict   :"
print predTestY[1200:1400]

Original  :
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3
 3 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3]
Predict   :
[1 2 3 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1 1 3 2 1 1 1 1 1 1 3 1 1 1 1 1
 1 1 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 1 2 1
 1 1 2 1 3 2 3 2 2 1 1 1 2 2 1 1 1 2 2 2 3 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 2 2 2 2 2 2 2 2 2 2 2 3 2 3 3 2 2 2 3
 2 2 3 2 2 2 2 3 2 3 3 3 3 3 3]


In [354]:
combined_data = []

for group in range(1, 11):
    data_raw = raw_loader.load_eval(str(group))

    for idx in data_raw.index:
        data = data_raw.ix[idx]
        
        try:
            data_sensor = raw_loader.load_sensor_data(data['File name'])
        except:
            continue
            
        situation = data[' Input situation']
        if situation == 'Sit':
            situation = 1
        elif situation == 'Stand':
            situation = 2
        elif situation == 'Walk':
            situation = 3
        else:
            situation = 0
        
        
        combined_data.append([situation,
                    data_sensor.acc_z.var(),
                    data_sensor.pitch.var(),
                    data_sensor.azim.var(),
                    data_sensor.roll.var(),
                    data_sensor.gyro_z.var(),
                    data_sensor.acc_z.mean(),
                    data_sensor.pitch.var(),
                    data_sensor.azim.mean(),
                    data_sensor.roll.mean(),
                    data_sensor.gyro_z.mean()])
        
df = DataFrame(combined_data, columns=['situation', 'acc_z', 'pitch', 'azim', 'roll', 'gyro_z', 'macc_z', 'mpitch', 'mazim', 'mroll', 'mgyro_z'])
df.to_csv('data/train1.csv')

In [355]:
train_data = read_csv('data/train1.csv')

trainY = train_data['situation'].values
trainX = train_data.drop('situation', 1)

In [356]:
test_data = []

for group in range(1, 3):
    data_raw = raw_loader.load_eval('test' + str(group))

    for idx in data_raw.index:
        data = data_raw.ix[idx]
        data_sensor = raw_loader.load_sensor_data(data['File name'])

        situation = data[' Input situation']
        if situation == 'Sit':
            situation = 1
        elif situation == 'Stand':
            situation = 2
        elif situation == 'Walk':
            situation = 3
        else:
            situation = 0

        age = data[' Age']

        test_data.append([situation,
                        data_sensor.acc_z.var(),
                        data_sensor.pitch.var(),
                        data_sensor.azim.var(),
                        data_sensor.roll.var(),
                        data_sensor.gyro_z.var(),
                        data_sensor.acc_z.mean(),
                        data_sensor.pitch.var(),
                        data_sensor.azim.mean(),
                        data_sensor.roll.mean(),
                        data_sensor.gyro_z.mean()])

df = DataFrame(test_data, columns=['situation', 'acc_z', 'pitch', 'azim', 'roll', 'gyro_z', 'macc_z', 'mpitch', 'mazim', 'mroll', 'mgyro_z'])
df.to_csv('data/test1.csv')

test_data = read_csv('data/test1.csv')

testY = test_data['situation'].values
testX = test_data.drop('situation', 1)

In [181]:
pandas.__version__

'0.13.1'